In [15]:
import torch
import transformer_lens
from src.game import Board, get_best_moves, apply_best_moves, play_game, State
import tqdm

In [2]:
boards = [Board()]
game_list = apply_best_moves(boards)

we're on the 1th loop!
we've got 9 boards, y'all!
we're on the 2th loop!
we've got 72 boards, y'all!
we're on the 3th loop!
we've got 456 boards, y'all!
we're on the 4th loop!
we've got 2016 boards, y'all!
we're on the 5th loop!
we've got 7584 boards, y'all!
we're on the 6th loop!
we've got 18720 boards, y'all!
we're on the 7th loop!
we've got 41472 boards, y'all!
we're on the 8th loop!
we've got 46080 boards, y'all!
we're on the 9th loop!
we've got 46080 boards, y'all!
we've got 46080 boards, y'all!


In [34]:
print(len(game_list))
print(game_list[-1].moves_played)
print(len({tuple(game.moves_played) for game in game_list}))
print(len(list(filter(lambda game: game.game_state == State.DRAW, game_list))))
a = all((board.game_state == State.DRAW) for board in game_list)


SyntaxError: invalid syntax (1709403322.py, line 5)

In [ ]:
board = Board()
print(get_best_moves(board.make_move(0).make_move(8)))

[1, 2, 3, 4, 5, 6, 7]


In [ ]:
for game in game_list:
    print(game.moves_played)

[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8]


In [ ]:
boards = [Board()]
boards = [board.make_move(move).moves_played for board in boards for move in get_best_moves(board)]


In [ ]:
print(boards)

[[0, 1, 2, 3, 4, 5, 6, 7, 8], [0, 1, 2, 3, 4, 5, 6, 7, 8], [0, 1, 2, 3, 4, 5, 6, 7, 8], [0, 1, 2, 3, 4, 5, 6, 7, 8], [0, 1, 2, 3, 4, 5, 6, 7, 8], [0, 1, 2, 3, 4, 5, 6, 7, 8], [0, 1, 2, 3, 4, 5, 6, 7, 8], [0, 1, 2, 3, 4, 5, 6, 7, 8], [0, 1, 2, 3, 4, 5, 6, 7, 8]]
